In [0]:
!pip install PyPDF2
!pip install fitz
!pip install PyMuPDF
!pip install gevent
!pip install flask_ngrok

     |████████████████████████████████| 81kB 3.5MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=be34bea9ab2caa76fc16f2baf4da0301ce641c929ca29fcdee60d08d380ac829
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 3.1MB 81kB/s 
     |████████████████████████████████| 5.5MB 45.3MB/s 
     |████████████████████████████████| 235kB 50.1MB/s 
     |████████████████████████████████| 573kB 45.6MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 430kB 31.4MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for pyxnat: filename=pyxnat-1.3-cp36-none-any.whl size=88337 sha256=50b2b116997bb7a937d047588d5b0c22fcef1b26607bb4736c1804974927bc6f
  Stored in directory: /root/.cache/pip/wheels/65/bb/9b/39ed8a8a3c3c720

In [0]:
!pip install flask_ngrok

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
model = VGG16(include_top=False, weights='imagenet')
model.save('/content/drive/My Drive/Deploy_Flask/models/model_vgg16.h5')

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# Enter PDF Source + Name                                       [done]
# Get Pdf info: Total Number of pages                           [done]
# Convert all pages to images : DocSplitter  -> Temp            [done]
# Insert Pages into Prediction Module                           [done]
# Get bookmark name and Bookmark                                [done]
# delete all the pictures                                       [done]
# Debug program                                                 [remaining]

#Starting Date & Time
from datetime import datetime
start_time = datetime.now()

#Flask import *
from flask import request
from flask import jsonify
from flask import Flask
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
from flask_ngrok import run_with_ngrok
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import load_model

# from keras.preprocessing.image import img_to_array
# from keras.models import load_model

import PyPDF2
import os
# from tensorflow import keras
# from keras.models import load_model
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras import optimizers
from keras.models import Sequential
# from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image
from tensorflow.python.keras.applications.vgg16 import VGG16
import sys
import fitz, shutil, re
import cv2
import keras.backend as K
import csv
from os import path


# Define a flask app
app = Flask(__name__)
run_with_ngrok(app)
# Model saved with Keras model.save()
#MODEL_PATH_VGG16 = '/content/drive/My Drive/Deploy_Flask/models/model_vgg16.h5'
#TRAIN_MODEL_PATH = '/content/drive/My Drive/Deploy_Flask/models/14_model.h5'
#top_model_weights_path = '/content/drive/My Drive/Deploy_Flask/models/14_model.h5'

# Load your trained model
#model =load_model(MODEL_PATH)
#model._make_predict_function()          # Necessary
print('Model loaded. Start serving...')

# You can also use pretrained model from Keras
# Check https://keras.io/applications/
#from keras.applications.resnet50 import ResNet50
#model = ResNet50(weights='imagenet')
#model.save('')


# @app.route("/path", methods=['GET', 'POST'])
@app.route("/path")
def file_path_getdata():
    # input_folder = request.form['npath']
    # output_folder = request.form['cpath']
    return render_template('index.html')


def get_data():
    #input folder & out put folder hadcode potion.
    input_folder = "/content/drive/My Drive/New_PDF/PDFNEWS/"
    output_folder = "/content/drive/My Drive/New_PDF/Output_path_folder/new/"
    return input_folder, output_folder


# dimensions of our images when loading.
#img_width, img_height = 500, 500
# filename = '/content/drive/My Drive/Share_File/Working/28_04_2020/ALT_CSV2.csv'
# csv  = []

#Creating the text file
#f=open("filename.txt","a+")

def Enter_Required_Data(File_Name,output_folder):
    # Message1 = "Enter PDF Source Path (End with Example: C:/User/Admin/Documents/):"
    # Message2 = "Enter PDF name (Example: Henry.pdf)"
    # print(Message1)
    # pdf_source = input()
    # print(Message2)
    # File_Name = input()
    print("\t",File_Name)
    File_Name = File_Name.split(".")[0]
    if path.exists(output_folder + File_Name):
      print ("File exist")
    else:
      print ("File not exist")
      os.mkdir(output_folder + File_Name)
    csv_filename = output_folder + File_Name +"/" +File_Name + "_CSV.csv"
    csv  = []
    print(csv_filename)
    #for filename in files
    return output_folder + File_Name + "/", csv_filename

def Pdf_data_Extractor(File_Name, pdf_source):
    #Png_conversion_Script = "/content/pdftopng"
    Pdf_Source_File_Name = pdf_source + File_Name

    reader = PyPDF2.PdfFileReader(Pdf_Source_File_Name)
    writer = PyPDF2.PdfFileWriter()

    doc = fitz.open(Pdf_Source_File_Name)
    print(doc)
    File_Name = File_Name.split('.')[0]
    if path.exists(pdf_source+"Temp"):
        Remove_Temp_Files(pdf_source)
    os.mkdir(pdf_source+"Temp")
    Total_Number_of_Pages = reader.getNumPages()

    for page in doc:  # iterate through the pages
        Temporary_Directory = pdf_source+"Temp/" + File_Name
        writer.addPage(reader.getPage(page.number))
        pix = page.getPixmap(alpha = False)  # render page to an image
        pix.writePNG(Temporary_Directory+"page-%i.png" % page.number)  # store image as a PNG
    return writer, Total_Number_of_Pages

def predict(image_path):
    # load the class_indices saved in the earlier step
    class_dictionary = np.load('/content/drive/My Drive/Share_File/Working/06_04_2020/14 DataSet/class_indices.npy',allow_pickle=True).item()

    num_classes = len(class_dictionary)

    #load and pre-process the image
    #orig = cv2.imread(image_path)
    print("[INFO] loading and preprocessing image...")
    #image = load_img(image_path, target_size=(500, 500, 3))
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)

    image = image / 255
    image = np.expand_dims(image, axis=0)

    #VGG16 loaded in the top
    model = VGG16(include_top=False, weights='imagenet')
    # model.save('/content/drive/My Drive/Deploy_Flask/models/model_vgg16.h5')
    #model =load_model(MODEL_PATH_VGG16)

    bottleneck_prediction = model.predict(image)
    #print("bottleneck_prediction = "+str(bottleneck_prediction))
    model = Sequential()
    model.add(Flatten(input_shape=bottleneck_prediction.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(125, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))   # to get probability prediction

    #model loaded in the top
    #top_model_weights_path = '/content/drive/My Drive/Share_File/Working/06_04_2020/14 DataSet/14_model.h5'
    top_model_weights_path = '/content/drive/My Drive/Deploy_Flask/models/14_model.h5'
    model.load_weights(top_model_weights_path)
    class_predicted = model.predict_classes(bottleneck_prediction)

    print("class_predicted"+str(class_predicted))
    output_y = model.predict(bottleneck_prediction)
    inID = class_predicted[0]
    print("class_predicted[0] = "+str(class_predicted[0]))
    inv_map = {v: k for k, v in class_dictionary.items()}

    label = inv_map[inID]
    print("Prediction: class ID: {}, Label: {}".format(inID, label))
    
    pos= -1
    count=0
    predict1 = {}
    maximum = max(output_y[0])
    for i in range(len(output_y[0])):
        predict1[inv_map[i]] = output_y[0][i]
        if maximum == output_y[0][i]:
          pos = i
          count = count + 1
        elif maximum * 0.222 < output_y[0][i]:
          count = count + 1
    print('Arg Maximum : ', maximum)
    print(predict1)
    print('sum of probability = {0}'.format(sum(output_y[0])))
    return label,count,pos, max(output_y[0]), sum(output_y[0])

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

def prediction_And_Bookmark(writer,Total_Number_of_Pages, source_name, file_name, output_path, csv_filename):
    f=open(output_path+file_name.split(".")[0]+".txt","a+")

    #CATEGORIES = os.listdir('/content/drive/My Drive/TBird/DOCS/train')
    
    # CATEGORIES.sort()
    CATEGORIT=[]
    class_dictionary = np.load('/content/drive/My Drive/Share_File/Working/06_04_2020/14 DataSet/class_indices.npy',allow_pickle=True).item()
    inv_map = {v: k for k, v in class_dictionary.items()}
    for g in range(0, len(class_dictionary)+1):
        if (g<14):
            CATEG =writer.addBookmark(inv_map[g], 0, color=(1,0,0), bold=True)
            CATEGORIT.append(CATEG)
        else:
            PARENT = writer.addBookmark("MISC DOCS", 0,color=(0,0,0), bold=True)

    List_of_images = os.listdir(source_name+"/Temp/")
    List_of_images.sort(key=natural_keys)
    print(List_of_images)
    b=0
    c=0
    import csv
    with open(csv_filename, 'w', newline='') as csvfile:
      fieldnames = ['Page_NO', 'Label', 'Predicted_CATEGORIES', 'Probability','Sum of Probability']
      writer_csv = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer_csv.writeheader()
      for Page_No, Img in enumerate(List_of_images):
          """Prediction Code Here returns pos"""
          b=b+1
          print(b)
          test_image = Image.open(source_name+"/Temp/"+Img)
          test_imag =source_name+"/Temp/"+Img
          print(test_imag)
          #test_image = process_img(test_image)
          layb, count, pos, out, OutSum = predict(test_imag)
          if count > 1:
            print("Not Found")
            writer.addBookmark("Not Found", Page_No, parent=PARENT)
          #elif CATEGORIT[pos]==('IndirectObject(605, 0)' or 'IndirectObject(589, 0)'):
          elif CATEGORIT[pos]== CATEGORIT[5]:
            print("Not Found",layb,CATEGORIT[pos])
            writer.addBookmark("Column", Page_No, parent=PARENT)
          elif CATEGORIT[pos]== CATEGORIT[13]:
            print("Not Found",layb,CATEGORIT[pos])
            writer.addBookmark("Write", Page_No, parent=PARENT)
          else:
            print(layb,CATEGORIT[pos])
            writer.addBookmark(layb, Page_No, parent=CATEGORIT[pos])
            c=c+1
            if c==1 :
              print("\n\n C is :",c)
              f.write("|{}|{}".format(Page_No+1,layb))

            else:
              print("\n\n C is :",c)
              f.write("|{}\n{}|{}".format(Page_No,Page_No+1,layb))

          writer_csv.writerow({'Page_NO':Page_No+1,'Label':layb,'Predicted_CATEGORIES':CATEGORIT[pos],'Probability':out,'Sum of Probability':OutSum})
          #writer.addBookmark(layb, Page_No, parent=CATEGORIT[pos])
          
          K.clear_session()
    
    f.close()
    sys.setrecursionlimit(2000)
    with open(output_path+file_name.split(".")[0]+"_ReBookmarked.pdf","wb") as out:
        writer.write(out)



def Remove_Temp_Files(pdf_source):
    for x in os.listdir(pdf_source+"Temp"):
        os.remove(pdf_source+"Temp/"+x)
    os.removedirs(pdf_source+"Temp")
    #os.removedirs(pdf_source+"Temp")
    print("Removed Temp Folder")

# @app.route('/predict', methods=['POST'])
#@app.route('/predict', methods=['GET', 'POST'])
# @app.route("/predict")
def main():
  input_folder, output_folder = get_data()
  #   #File_Name, pdf_source = Enter_Required_Data()
  # print("Hello Error","\n",input_folder,"\n",output_folder)
  for filename in os.listdir(input_folder):
    if filename.endswith('.pdf'):
      File_Name = filename
      pdf_source = input_folder
      output_path,csv_filename = Enter_Required_Data(filename, output_folder)
      Source_Name = pdf_source + File_Name

      writer, Total_Number_of_Pages = Pdf_data_Extractor(File_Name, pdf_source)

      prediction_And_Bookmark(writer, Total_Number_of_Pages, pdf_source,File_Name, output_path, csv_filename)

      Remove_Temp_Files(pdf_source)

  #return "</center><h1>Running Flask on Google Colab!</br> Complited...</h1></center>"

@app.route("/predict")
def home():
    return "<h1><center>Running Flask on Google Colab!<br><br><u><b>Auto Indexing...</b></u></center></h1>"


if __name__ == "__main__":
    import tensorflow as tf
    app.run()
    #run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
    #main()
    try:
      main()
    except:
      print("Something went wrong")
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

Model loaded. Start serving...
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Using TensorFlow backend.
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://01dcc292ee96.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Jun/2020 20:04:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [08/Jun/2020 20:04:41] "GET /favicon.ico HTTP/1.1" 404 -
[2020-06-08 20:04:47,736] ERROR in app: Exception on /path [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoi

In [0]:
from flask_ngrok import run_with_ngrok
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

import tensorflow
# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras.models import load_model

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Define a flask app
app = Flask(__name__)
run_with_ngrok(app)
# Model saved with Keras model.save()
MODEL_PATH = '/content/Deployment-Deep-Learning-Model/models/model_resnet.h5'

# Load your trained model
model =load_model(MODEL_PATH)
#model._make_predict_function()          # Necessary
# print('Model loaded. Start serving...')

# You can also use pretrained model from Keras
# Check https://keras.io/applications/
#from keras.applications.resnet50 import ResNet50
#model = ResNet50(weights='imagenet')
#model.save('')
print('Model loaded. Check http://127.0.0.1:5000/')


def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x, mode='caffe')

    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # #def upload_files():
        # from google.colab import files
        # uploaded = files.upload()
        # for k, v in uploaded.items():
        #   open(k, 'wb').write(v)
        # return list(uploaded.keys())


        # Save the file to ./uploads
        # basepath = os.path.dirname(__file__)
        # file_path = os.path.join(
        #     basepath, 'uploads', secure_filename(f.filename))
        #f.save(file_path='/content/Deployment-Deep-Learning-Model/uploads/cat.jpeg')
        file_path='/content/Deployment-Deep-Learning-Model/uploads/cat.jpeg'

        # Make prediction
        preds = model_predict(file_path, model)

        # Process your result for human
        # pred_class = preds.argmax(axis=-1)            # Simple argmax
        pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
        result = str(pred_class[0][0][1])               # Convert to string
        return result
    return None


if __name__ == '__main__':
    app.run()
    # app.run(debug=True)

Model loaded. Check http://127.0.0.1:5000/
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ced02215575a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Jun/2020 16:37:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 16:37:31] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 16:37:32] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 16:37:44] "GET /favicon.ico HTTP/1.1" 404 -


40960/35363 [==================================] - 0s 0us/step


127.0.0.1 - - [08/Jun/2020 16:38:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 16:38:52] "POST /predict HTTP/1.1" 200 -
[2020-06-08 16:39:17,375] ERROR in app: Exception on /predict [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [08/Jun/2020 16:39:17] "GET /predict HTTP/1.1" 500 -
127.0.0.1 - - [

In [4]:
import os,shutil
ddir = '/content/drive/My Drive/Deploy_Flask/models/14_model.h5'
sdir = '/content/drive/My Drive/Share_File/Working/06_04_2020/14 DataSet/14_model.h5'
shutil.copy(sdir,ddir)

'/content/drive/My Drive/Deploy_Flask/models/14_model.h5'